# Res-net model

In [2]:
# ==== Standard Libraries ====
import os, time, warnings

# ==== Scientific & Data Handling ====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ==== PyTorch Core ====
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from torchvision.models import resnet18

# ==== ML & Evaluation ====
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import (
    confusion_matrix, classification_report, f1_score,
    accuracy_score, balanced_accuracy_score, roc_auc_score, recall_score
)
from sklearn.utils import resample
from statsmodels.stats.proportion import proportions_ztest

# ==== Utilities ====
from tqdm import tqdm

# ==== Reproducibility ====
SEED = 42

# ==== Device Handling (DirectML + fallback CPU) ====
try:
    import torch_directml
    DEVICE = torch_directml.device()
    print("Using DirectML device:", DEVICE)
except Exception as e:
    DEVICE = torch.device("cpu")
    print("DirectML no disponible. Usando CPU:", e)

# ==== Warnings ====
warnings.filterwarnings('ignore')

Using DirectML device: privateuseone:0


In [3]:
CSV_PATH = '../Data/Tabular.csv'
FOLDER_PATH = r"C:\Users\usuario\MRI\IMAGES_npy"

OUTPUT_DIR = "../Models_Output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
# [] Atributos de las imégenes
shapes, means, stds, size = [], [], [], []

for f in os.listdir(FOLDER_PATH):
    if f.endswith(".npy"):
        img = np.load(os.path.join(FOLDER_PATH, f))
        shapes.append(img.shape)
        means.append(img.mean())
        stds.append(img.std())
        size.append(img.size)

# Contar shapes únicos
shapes_unicos = set(shapes)
size_unicos = set(size)
print("Shapes únicos encontrados:", shapes_unicos)
print("Total de imágenes:", len(shapes))
print(f"Media global promedio: {np.mean(means):.4f}")
print(f"Desviación global promedio: {np.mean(stds):.4f}")
print(f"Size únicos: {size_unicos}")

Shapes únicos encontrados: {(160, 192, 192)}
Total de imágenes: 220
Media global promedio: 0.0000
Desviación global promedio: 1.0000
Size únicos: {5898240}


In [5]:
# LOAD CSV
# ============================
df = pd.read_csv(CSV_PATH, dtype={'sujeto_id': str})
df = df.dropna(subset=['is_dementia'])

df['sujeto_id'] = df['sujeto_id'].astype(str).str.strip()
df['imagen_id'] = df['imagen_id'].astype(str).str.strip()

# FULL FILE PATH
def build_path(row):
    filename = f"{row['sujeto_id']}_{row['imagen_id']}.npy"
    return os.path.join(FOLDER_PATH, filename)

df['file_path'] = df.apply(build_path, axis=1)

df = df[['file_path', 'sujeto_id', 'is_dementia']].rename(
    columns={'is_dementia': 'label'}
)


In [6]:
# 1. Distribución básica de is_dementia

class_distribution = df['label'].value_counts().sort_index()
print("=== DISTRIBUCIÓN DE CLASES ===")
print(f"Clase 0 (No Dementia): {class_distribution.get(0, 0)} imágenes - {class_distribution.get(0, 0)/len(df)*100:.2f}%")
print(f"Clase 1 (Dementia): {class_distribution.get(1, 0)} imágenes - {class_distribution.get(1, 0)/len(df)*100:.2f}%")
class_distribution = df['label'].value_counts().sort_index()
N = len(df)
count_dementia = class_distribution.get(1, 0)
expected_proportion = 0.5 
z_stat, p_value = proportions_ztest(count=count_dementia, 
                                    nobs=N, 
                                    value=expected_proportion, 
                                    alternative='two-sided')
print(f"Estadístico Z: {z_stat:.4f}")
print(f"Valor p: {p_value:.4e}")
# Conclusión
alpha = 0.05
if p_value < alpha:
    print(f"\nConclusión: Se rechaza H0. La proporción de la Clase 1 ({count_dementia/N*100:.2f}%) es SIGNIFICATIVAMENTE diferente de {expected_proportion*100:.0f}%.")
else:
    print(f"\nConclusión: No se rechaza H0. No hay evidencia suficiente para decir que la proporción es diferente de {expected_proportion*100:.0f}%.")

=== DISTRIBUCIÓN DE CLASES ===
Clase 0 (No Dementia): 158 imágenes - 71.82%
Clase 1 (Dementia): 62 imágenes - 28.18%
Estadístico Z: -7.1933
Valor p: 6.3244e-13

Conclusión: Se rechaza H0. La proporción de la Clase 1 (28.18%) es SIGNIFICATIVAMENTE diferente de 50%.


In [7]:
def group_stratified_split(records):
    """
    Split estratificado por SUJETO:
        Train 60%
        Val   20%
        Test  20%
    Sin fuga de información.
    """

    df_local = pd.DataFrame(records)

    # 1. Etiqueta por sujeto (promedio -> entero)
    subj_lab = df_local.groupby("sujeto_id")["label"].agg(lambda x: int(round(x.mean())))
    subjects = subj_lab.index.to_list()
    y = subj_lab.values

    # -----------------------------
    # 2. Train (60%) vs Resto (40%)
    # -----------------------------
    gss1 = GroupShuffleSplit(n_splits=1, train_size=0.6, random_state=SEED)
    train_idx, rest_idx = next(gss1.split(subjects, y, groups=subjects))

    train_subj = [subjects[i] for i in train_idx]
    rest_subj  = [subjects[i] for i in rest_idx]

    # -----------------------------
    # 3. Val (20%) vs Test (20%) dentro del 40%
    # -----------------------------
    rest_labels = [subj_lab[s] for s in rest_subj]

    val_subj, test_subj = train_test_split(
        rest_subj,
        test_size=0.5,
        random_state=SEED,
        stratify=rest_labels
    )

    # -----------------------------
    # 4. Regresar DataFrames
    # -----------------------------
    def pick(subjects_list):
        return df_local[df_local["sujeto_id"].isin(subjects_list)].reset_index(drop=True)

    return pick(train_subj), pick(val_subj), pick(test_subj)


In [8]:
# [PASO 2] Construcción del DataSet MRI 2.5D
# ---------------------------

class MRI2p5DDataset(Dataset):
    def __init__(self, df, n_slices, target_size=(224,224), augment=False):
        """
        df: DataFrame con columnas 'file_path' y 'label'
        n_slices: número de cortes a tomar por volumen
        target_size: tamaño HxW de las imágenes
        augment: aplicar data augmentation
        """
        self.records = df.to_dict("records")
        self.n_slices = n_slices
        self.target_size = target_size
        self.augment = augment

        # Transforms
        self.augment_tf = T.Compose([
            T.RandomRotation(10),
            T.RandomResizedCrop(target_size, scale=(0.9,1.0))
        ])
        self.base_tf = T.Compose([
            T.Resize(target_size),
            T.ToTensor()
        ])
        
    def __len__(self):
        return len(self.records)
    
    def __getitem__(self, idx):
        rec = self.records[idx]
        vol = np.load(rec["file_path"])  # (D,H,W) 3D array, Cortes-Altura-Ancho

        # Elegir n_slices uniformemente
        D = vol.shape[0]
        idxs = np.linspace(0, D-1, self.n_slices).astype(int)
        slices = vol[idxs]  # (n_slices,H,W)

        # Convertir cada slice a tensor
        imgs = []
        for s in slices:
            # Normalizar a [0,255] y convertir a PIL
            s_img = ((s - s.min())/(s.max()-s.min()+1e-6) * 255).astype(np.uint8)
            img = Image.fromarray(s_img).convert("L")  # 1 canal
            if self.augment:
                img = self.augment_tf(img)
            img = self.base_tf(img)  # (1,H,W)
            imgs.append(img)
        
        # Apilar slices como canales -> (n_slices,H,W)
        input_tensor = torch.cat(imgs, dim=0)
        
        label = torch.tensor(rec["label"], dtype=torch.float32)
        return input_tensor, label

In [9]:
# [PASO 3] Modelo ResNet18 MRI 2.5D
# ---------------------------

class ResNet18_2p5D(nn.Module):
    """
    Modelo 2.5D basado en ResNet18 para clasificación binaria de volúmenes 3D (MRI). 
    
    Idea clave:
    - Cada volumen 3D se divide en 'n_slices' cortes (slices) uniformes.
    - Los slices se apilan como canales de una imagen 2D de entrada.
    - La ResNet18 procesa todos los slices simultáneamente como una "imagen 2D con n_slices canales".
    - La salida es un logit único por volumen, útil para predecir probabilidad de progresión de Alzheimer.

    Parámetros
    ----------
    n_slices : int
        Número de cortes (slices) que se apilan como canales de entrada.
        Por defecto: 32.
    """

    def __init__(self, n_slices):
        super().__init__()

        # Base ResNet18 sin pesos preentrenados
        self.model = resnet18(weights=None)
        self.model.conv1 = nn.Conv2d(
            in_channels=n_slices,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        self.model.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(512,1)
        )

    def forward(self, x):
        """
        Forward pass del modelo.

        Parámetros
        ----------
        x : torch.Tensor
            Tensor de entrada de forma (B, n_slices, H, W)
            donde B = batch size, n_slices = número de slices apilados,
            H y W = dimensiones espaciales de cada slice.

        Retorna
        -------
        logits : torch.Tensor
            Tensor de forma (B,), logit único por volumen.
            Este logit se puede pasar a sigmoid para obtener probabilidad.
        """
        # x: (B, n_slices, H, W)
        logits = self.model(x)  # ResNet18 produce (B,1)
        return logits.squeeze(1)  # Redimensiona a (B,)


In [10]:
# [PASO 4] Entrenamiento por epoca y evaluacion
# ---------------------------

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()                    # Coloca el modelo en modo entrenamiento
    total_loss = 0
    total = 0
    pbar = tqdm(loader, desc="Train", leave=False)
    
    for X, y in pbar:
        X, y = X.to(device), y.to(device)  # Enviar batch a GPU/CPU
        optimizer.zero_grad()              # Reiniciar gradientes

        logits = model(X)                  # Forward
        loss = criterion(logits, y)        # Calcular pérdida
        loss.backward()                     # Backpropagation
        optimizer.step()                    # Actualizar pesos

        total_loss += loss.item() * y.size(0)  # Acumular pérdida ponderada por batch
        total += y.size(0)
        pbar.set_postfix({"batch_loss": loss.item()})  # Mostrar pérdida por batch

    return total_loss / total  # Pérdida promedio por sample

def evaluate(model, val_loader, device):
    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.float().to(device)

            logits = model(images)
            prob = torch.sigmoid(logits).cpu().numpy().flatten()
            pred = (prob >= 0.5).astype(int)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(pred)
            all_probs.extend(prob)

    acc = accuracy_score(all_labels, all_preds)
    bal_acc = balanced_accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    recall = recall_score(all_labels, all_preds)

    return acc, bal_acc, auc, recall


In [11]:
def prepare_dataloaders(df, batch_size, n_slices, target_size=(224,224)):
    records = df.to_dict("records")

    # -------- SPLIT POR SUJETO (60/20/20) --------
    train_df, val_df, test_df = group_stratified_split(records)

    #print(f"Dimensiones: Train {len(train_df)} | Val {len(val_df)} | Test {len(test_df)}")

    # ========== CALCULAR CLASE PREDOMINANTE POR SUJETO ==========
    subj_lab = train_df.groupby("sujeto_id")["label"].agg(lambda x: int(round(x.mean())))

    #print("\n=== Etiqueta predominante por sujeto (Train) ===")
    #print(subj_lab.value_counts())

    # Sujetos mayoritarios/minoritarios según la etiqueta predominante
    maj_subj = subj_lab[subj_lab == 0].index.tolist()
    min_subj = subj_lab[subj_lab == 1].index.tolist()

    #print(f"Sujetos clase 0 (majority): {len(maj_subj)}")
    #print(f"Subjects clase 1 (minority): {len(min_subj)}")

    # ========== UPSAMPLING A NIVEL DE SUJETO ==========
    min_subj_upsampled = resample(
        min_subj,
        replace=True,
        n_samples=len(maj_subj),
        random_state=42
    )

    #print("\nSujetos repetidos en oversampling:")
    #print(pd.Series(min_subj_upsampled).value_counts())

    # ========== RECONSTRUIR TRAIN BALANCEADO SIN MODIFICAR VISITAS ==========
    train_df_bal = pd.concat([
        train_df[train_df["sujeto_id"].isin(maj_subj)],
        train_df[train_df["sujeto_id"].isin(min_subj_upsampled)]
    ]).sample(frac=1, random_state=42)

    #print("\n=== Distribución después de oversampling (por imagen) ===")
    #print(train_df_bal["label"].value_counts())

    # ========== DATASETS ==========
    train_dataset = MRI2p5DDataset(train_df_bal, n_slices=n_slices,
                                   target_size=target_size, augment=True)
    val_dataset   = MRI2p5DDataset(val_df, n_slices=n_slices,
                                   target_size=target_size, augment=False)
    test_dataset  = MRI2p5DDataset(test_df, n_slices=n_slices,
                                   target_size=target_size, augment=False)

    # ========== DATALOADERS ==========
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size)

    #print(f"\nTrain Loader: {len(train_loader.dataset)} ejemplos, {len(train_loader)} batches")
    #print(f"Validation Loader: {len(val_loader.dataset)} ejemplos, {len(val_loader)} batches")
    #print(f"Test Loader: {len(test_loader.dataset)} ejemplos, {len(test_loader)} batches")

    return train_loader, val_loader, test_loader


In [12]:
# [PASO 6] Modulo de entrenamiento
# ---------------------------
def train_model(
        model, train_loader, val_loader, device, 
        epochs, lr, weight_decay, early_stopping_patience, model_ej):
    """
    Entrena el modelo y retorna listas de pérdidas y métricas.
    """
    print(f"\n⏺️ Épocas {epochs} | LR-WD {lr}-{ weight_decay} | Patience {early_stopping_patience}")

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    train_losses, val_recalls, val_aucs, val_bac = [], [], [], []
    best_bal_acc = 0
    patience_counter = 0

    print("▶️ Start train")
    time_all = time.time()

    for ep in range(epochs):
        time_ep = time.time()
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        time_train = time.time() - time_ep
        acc, bal_acc, auc, recall = evaluate(model, val_loader, device)
        
        train_losses.append(train_loss)
        val_recalls.append(recall)
        val_aucs.append(auc)
        val_bac.append(bal_acc)

        #print(f"Epoch {ep+1}/{epochs} | TrainLoss={train_loss:.4f} ||"
           #   f"BAC ={bal_acc:.4f} | Recall ={recall:.4f} | "
           #   f"AUC={auc:.4f} | Time={time_train/60:.2f}")
        # Scheduler
        scheduler.step(train_loss)

        # EARLY STOPPING usando RECALL DE LA CLASE 1
        if bal_acc > best_bal_acc:  # Considera ambas clases
            best_bal_acc = bal_acc
            patience_counter = 0
            model_path = f"../Models_Output/{model_ej}_{epochs}_{ep}_{early_stopping_patience}.pth"
            torch.save(model.state_dict(), model_path)
            print(f">> Mejor modelo guardado"
                  f"Epoch {ep+1}/{epochs} | TrainLoss={train_loss:.4f} ||"
                  f"BAC ={bal_acc:.4f} | Recall ={recall:.4f} | "
                  f"AUC={auc:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f">> Early stopping activado {ep}")
                break

    
    print(f"Tiempo total de entrenamiento: {(time.time() - time_all)/60:.2f} minutos")
    return train_losses, val_bac, val_aucs, model_path

In [13]:
def evaluate_final(model, test_loader, device, model_path):
    """
    Evalúa el mejor modelo guardado y calcula métricas relevantes,
    incluyendo sensibilidad (recall de clase positiva) y especificidad.
    """
    print(f"Best model: {model_path}")
    model.load_state_dict(torch.load(model_path))
    model.eval()

    all_preds, all_labels = [], []

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).float()
            all_preds.append(preds.cpu())
            all_labels.append(y.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    acc = (all_preds == all_labels).float().mean().item()
    bal_acc = balanced_accuracy_score(all_labels, all_preds)

    try:
        auc = roc_auc_score(all_labels, all_preds)
    except ValueError:
        auc = float('nan')

    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"\n TEST =====")

    print(f"BAC = {bal_acc:.4f} | AUC = {auc:.4f} | recall = {recall:.4f}  | f1 = {f1:.4f}")
    # Matriz de confusión
    cm = confusion_matrix(all_labels, all_preds)
    print("\nMatriz de confusión:")
    print(cm)

    # Reporte de clasificación
    report = classification_report(all_labels, all_preds, target_names=["No Dementia", "Dementia"])
    print("\nReporte de clasificación:")
    print(report)

    return acc, bal_acc, auc, recall, f1, cm, report


In [14]:
# ---------------------------
# MAIN
# ---------------------------
def main(df, device, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO):
    print(F"✳️ Entrenando modelo {MODELO}")
    train_loader, val_loader, test_loader = prepare_dataloaders(df, BATCH, n_slices)

    model = ResNet18_2p5D(n_slices).to(device)
    
    train_losses, val_bac, val_aucs, model_path = train_model(
        model, train_loader, val_loader, device, 
        EPOCHS, LR, WEIGHT, EARLY, MODELO)
        
    # Evaluación final
    evaluate_final(model, test_loader, device, model_path)


-----------------

In [15]:
EPOCHS = 20 
EARLY = 3

In [16]:
import itertools

# Definir los rangos de LR y WEIGHT
LR_RANGE = [5e-3, 1e-3, 5e-4, 1e-4]
WEIGHT_RANGE = [5e-3, 1e-3, 5e-4, 1e-4]

# Crear todas las combinaciones posibles (16 en total)
grid = list(itertools.product(LR_RANGE, WEIGHT_RANGE))

## 16

### 4

In [32]:
# Ejecutar 1
# ---------------------------
n_slices = 16
BATCH = 4
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_16_4

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.1083 ||BAC =0.5000 | Recall =0.0000 | AUC=0.4345


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.28 minutos
Best model: ../Models_Output/Ejex_1_16_4_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53

✳️ Entrenando modelo Ejex_2_16_4

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.8208 ||BAC =0.5000 | Recall =0.0000 | AUC=0.4103


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.55 minutos
Best model: ../Models_Output/Ejex_2_16_4_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53

✳️ Entrenando modelo Ejex_3_16_4

⏺️ Épocas 20 | LR-WD 0.005-0.0005 | Patience 3
▶️ Start train


...

>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.6587 ||BAC =0.5000 | Recall =0.0000 | AUC=0.4069


>> Mejor modelo guardadoEpoch 3/20 | TrainLoss=0.6222 ||BAC =0.5172 | Recall =1.0000 | AUC=0.5828


>> Mejor modelo guardadoEpoch 6/20 | TrainLoss=0.5233 ||BAC =0.5362 | Recall =0.9000 | AUC=0.4862


>> Mejor modelo guardadoEpoch 8/20 | TrainLoss=0.4763 ||BAC =0.5586 | Recall =0.6000 | AUC=0.5310


>> Early stopping activado 10
Tiempo total de entrenamiento: 4.44 minutos
Best model: ../Models_Output/Ejex_16_16_4_20_7_3.pth

 TEST =====
BAC = 0.5146 | AUC = 0.5146 | recall = 0.8000  | f1 = 0.1739

Matriz de confusión:
[[11 37]
 [ 1  4]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.92      0.23      0.37        48
    Dementia       0.10      0.80      0.17         5

    accuracy                           0.28        53
   macro avg       0.51      0.51      0.27        53
weighted avg       0.84      0.28      0.35        53



### 8

In [33]:
# Ejecutar 2
# ---------------------------
n_slices = 16
BATCH = 8
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_16_8

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.1656 ||BAC =0.5000 | Recall =1.0000 | AUC=0.4500


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.15 minutos
Best model: ../Models_Output/Ejex_1_16_8_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 1.0000  | f1 = 0.1724

Matriz de confusión:
[[ 0 48]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.00      0.00      0.00        48
    Dementia       0.09      1.00      0.17         5

    accuracy                           0.09        53
   macro avg       0.05      0.50      0.09        53
weighted avg       0.01      0.09      0.02        53

✳️ Entrenando modelo Ejex_2_16_8

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.9840 ||BAC =0.5793 | Recall =0.4000 | AUC=0.5862


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.16 minutos
Best model: ../Models_Output/Ejex_2_16_8_20_0_3.pth

 TEST =====
BAC = 0.5271 | AUC = 0.5271 | recall = 0.2000  | f1 = 0.1538

Matriz de confusión:
[[41  7]
 [ 4  1]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      0.85      0.88        48
    Dementia       0.12      0.20      0.15         5

    accuracy                           0.79        53
   macro avg       0.52      0.53      0.52        53
weighted avg       0.84      0.79      0.81        53

✳️ Entrenando modelo Ejex_3_16_8

⏺️ Épocas 20 | LR-WD 0.005-0.0005 | Patience 3
▶️ Start train


...

>> Mejor modelo guardadoEpoch 4/20 | TrainLoss=0.5017 ||BAC =0.5914 | Recall =0.7000 | AUC=0.4862


>> Early stopping activado 6
Tiempo total de entrenamiento: 2.06 minutos
Best model: ../Models_Output/Ejex_15_16_8_20_3_3.pth

 TEST =====
BAC = 0.3250 | AUC = 0.3250 | recall = 0.4000  | f1 = 0.0930

Matriz de confusión:
[[12 36]
 [ 3  2]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.80      0.25      0.38        48
    Dementia       0.05      0.40      0.09         5

    accuracy                           0.26        53
   macro avg       0.43      0.33      0.24        53
weighted avg       0.73      0.26      0.35        53

✳️ Entrenando modelo Ejex_16_16_8

⏺️ Épocas 20 | LR-WD 0.0001-0.0001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.6770 ||BAC =0.4655 | Recall =0.0000 | AUC=0.3310


>> Mejor modelo guardadoEpoch 2/20 | TrainLoss=0.6424 ||BAC =0.5000 | Recall =0.0000 | AUC=0.4138


>> Early stopping activado 4
Tiempo total de entrenamiento: 1.46 minutos
Best model: ../Models_Output/Ejex_16_16_8_20_1_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53



### 16

In [34]:
n_slices = 16
BATCH = 16
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_16_16

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.1752 ||BAC =0.5000 | Recall =0.0000 | AUC=0.5000


>> Early stopping activado 3
Tiempo total de entrenamiento: 0.91 minutos
Best model: ../Models_Output/Ejex_1_16_16_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53

✳️ Entrenando modelo Ejex_2_16_16

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.2207 ||BAC =0.5000 | Recall =0.0000 | AUC=0.5000


>> Mejor modelo guardadoEpoch 4/20 | TrainLoss=0.6486 ||BAC =0.5690 | Recall =1.0000 | AUC=0.3759


...

>> Mejor modelo guardadoEpoch 4/20 | TrainLoss=0.5257 ||BAC =0.5155 | Recall =0.1000 | AUC=0.4172


>> Early stopping activado 6
Tiempo total de entrenamiento: 1.67 minutos
Best model: ../Models_Output/Ejex_15_16_16_20_3_3.pth

 TEST =====
BAC = 0.6958 | AUC = 0.6958 | recall = 0.6000  | f1 = 0.3333

Matriz de confusión:
[[38 10]
 [ 2  3]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.95      0.79      0.86        48
    Dementia       0.23      0.60      0.33         5

    accuracy                           0.77        53
   macro avg       0.59      0.70      0.60        53
weighted avg       0.88      0.77      0.81        53

✳️ Entrenando modelo Ejex_16_16_16

⏺️ Épocas 20 | LR-WD 0.0001-0.0001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.7066 ||BAC =0.5000 | Recall =1.0000 | AUC=0.5379


>> Mejor modelo guardadoEpoch 3/20 | TrainLoss=0.5930 ||BAC =0.5052 | Recall =0.7000 | AUC=0.4690


>> Early stopping activado 5
Tiempo total de entrenamiento: 1.45 minutos
Best model: ../Models_Output/Ejex_16_16_16_20_2_3.pth

 TEST =====
BAC = 0.5563 | AUC = 0.5563 | recall = 0.8000  | f1 = 0.1905

Matriz de confusión:
[[15 33]
 [ 1  4]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.94      0.31      0.47        48
    Dementia       0.11      0.80      0.19         5

    accuracy                           0.36        53
   macro avg       0.52      0.56      0.33        53
weighted avg       0.86      0.36      0.44        53



## 32

### 4

In [35]:
n_slices = 32
BATCH = 4
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_32_4

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.9322 ||BAC =0.5172 | Recall =1.0000 | AUC=0.4172


>> Early stopping activado 3
Tiempo total de entrenamiento: 2.03 minutos
Best model: ../Models_Output/Ejex_1_32_4_20_0_3.pth

 TEST =====
BAC = 0.5104 | AUC = 0.5104 | recall = 1.0000  | f1 = 0.1754

Matriz de confusión:
[[ 1 47]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       1.00      0.02      0.04        48
    Dementia       0.10      1.00      0.18         5

    accuracy                           0.11        53
   macro avg       0.55      0.51      0.11        53
weighted avg       0.91      0.11      0.05        53

✳️ Entrenando modelo Ejex_2_32_4

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.8484 ||BAC =0.4500 | Recall =0.9000 | AUC=0.6207


>> Mejor modelo guardadoEpoch 2/20 | TrainLoss=0.7044 ||BAC =0.5000 | Recall =0.0000 | AUC=0.5345


...

>> Mejor modelo guardadoEpoch 4/20 | TrainLoss=0.6110 ||BAC =0.5172 | Recall =1.0000 | AUC=0.5138


>> Early stopping activado 6
Tiempo total de entrenamiento: 13.51 minutos
Best model: ../Models_Output/Ejex_15_32_4_20_3_3.pth

 TEST =====
BAC = 0.5104 | AUC = 0.5104 | recall = 1.0000  | f1 = 0.1754

Matriz de confusión:
[[ 1 47]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       1.00      0.02      0.04        48
    Dementia       0.10      1.00      0.18         5

    accuracy                           0.11        53
   macro avg       0.55      0.51      0.11        53
weighted avg       0.91      0.11      0.05        53

✳️ Entrenando modelo Ejex_16_32_4

⏺️ Épocas 20 | LR-WD 0.0001-0.0001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.7113 ||BAC =0.3845 | Recall =0.7000 | AUC=0.3483


>> Mejor modelo guardadoEpoch 2/20 | TrainLoss=0.6481 ||BAC =0.5000 | Recall =0.0000 | AUC=0.4483


>> Early stopping activado 4
Tiempo total de entrenamiento: 2.43 minutos
Best model: ../Models_Output/Ejex_16_32_4_20_1_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53



### 8

In [16]:
n_slices = 32
BATCH = 8
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_32_8

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.9320 ||BAC =0.5000 | Recall =0.0000 | AUC=0.4414


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.74 minutos
Best model: ../Models_Output/Ejex_1_32_8_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53

✳️ Entrenando modelo Ejex_2_32_8

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.8196 ||BAC =0.5000 | Recall =1.0000 | AUC=0.5000


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.64 minutos
Best model: ../Models_Output/Ejex_2_32_8_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 1.0000  | f1 = 0.1724

Matriz de confusión:
[[ 0 48]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.00      0.00      0.00        48
    Dementia       0.09      1.00      0.17         5

    accuracy                           0.09        53
   macro avg       0.05      0.50      0.09        53
weighted avg       0.01      0.09      0.02        53

✳️ Entrenando modelo Ejex_3_32_8

⏺️ Épocas 20 | LR-WD 0.005-0.0005 | Patience 3
▶️ Start train


...

>> Mejor modelo guardadoEpoch 2/20 | TrainLoss=0.6714 ||BAC =0.5000 | Recall =0.0000 | AUC=0.3655


>> Mejor modelo guardadoEpoch 3/20 | TrainLoss=0.5951 ||BAC =0.5328 | Recall =0.1000 | AUC=0.4414


>> Mejor modelo guardadoEpoch 4/20 | TrainLoss=0.6132 ||BAC =0.5500 | Recall =0.1000 | AUC=0.4724


>> Mejor modelo guardadoEpoch 5/20 | TrainLoss=0.5308 ||BAC =0.5690 | Recall =1.0000 | AUC=0.5862


>> Early stopping activado 7
Tiempo total de entrenamiento: 3.68 minutos
Best model: ../Models_Output/Ejex_16_32_8_20_4_3.pth

 TEST =====
BAC = 0.5625 | AUC = 0.5625 | recall = 1.0000  | f1 = 0.1923

Matriz de confusión:
[[ 6 42]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       1.00      0.12      0.22        48
    Dementia       0.11      1.00      0.19         5

    accuracy                           0.21        53
   macro avg       0.55      0.56      0.21        53
weighted avg       0.92      0.21      0.22        53



### 16

In [17]:
n_slices = 32
BATCH = 16
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_32_16

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.0539 ||BAC =0.5000 | Recall =0.0000 | AUC=0.5000


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.57 minutos
Best model: ../Models_Output/Ejex_1_32_16_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53

✳️ Entrenando modelo Ejex_2_32_16

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.9456 ||BAC =0.5000 | Recall =0.0000 | AUC=0.5000


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.57 minutos
Best model: ../Models_Output/Ejex_2_32_16_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53

✳️ Entrenando modelo Ejex_3_32_16

⏺️ Épocas 20 | LR-WD 0.005-0.0005 | Patience 3
▶️ Start train


...

>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.7121 ||BAC =0.5000 | Recall =1.0000 | AUC=0.5241


>> Mejor modelo guardadoEpoch 4/20 | TrainLoss=0.6056 ||BAC =0.5224 | Recall =0.7000 | AUC=0.4828


>> Early stopping activado 6
Tiempo total de entrenamiento: 2.76 minutos
Best model: ../Models_Output/Ejex_15_32_16_20_3_3.pth

 TEST =====
BAC = 0.5604 | AUC = 0.5604 | recall = 0.6000  | f1 = 0.1935

Matriz de confusión:
[[25 23]
 [ 2  3]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.93      0.52      0.67        48
    Dementia       0.12      0.60      0.19         5

    accuracy                           0.53        53
   macro avg       0.52      0.56      0.43        53
weighted avg       0.85      0.53      0.62        53

✳️ Entrenando modelo Ejex_16_32_16

⏺️ Épocas 20 | LR-WD 0.0001-0.0001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.6589 ||BAC =0.5000 | Recall =1.0000 | AUC=0.4724


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.56 minutos
Best model: ../Models_Output/Ejex_16_32_16_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 1.0000  | f1 = 0.1724

Matriz de confusión:
[[ 0 48]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.00      0.00      0.00        48
    Dementia       0.09      1.00      0.17         5

    accuracy                           0.09        53
   macro avg       0.05      0.50      0.09        53
weighted avg       0.01      0.09      0.02        53



## 64

### 4

In [18]:
n_slices = 64
BATCH = 4
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_64_4

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.8776 ||BAC =0.6655 | Recall =0.4000 | AUC=0.5414


>> Early stopping activado 3
Tiempo total de entrenamiento: 2.46 minutos
Best model: ../Models_Output/Ejex_1_64_4_20_0_3.pth

 TEST =====
BAC = 0.3750 | AUC = 0.3750 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[36 12]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.88      0.75      0.81        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.68        53
   macro avg       0.44      0.38      0.40        53
weighted avg       0.80      0.68      0.73        53

✳️ Entrenando modelo Ejex_2_64_4

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.9633 ||BAC =0.5362 | Recall =0.9000 | AUC=0.5276


>> Early stopping activado 3
Tiempo total de entrenamiento: 2.60 minutos
Best model: ../Models_Output/Ejex_2_64_4_20_0_3.pth

 TEST =====
BAC = 0.4417 | AUC = 0.4417 | recall = 0.8000  | f1 = 0.1509

Matriz de confusión:
[[ 4 44]
 [ 1  4]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.80      0.08      0.15        48
    Dementia       0.08      0.80      0.15         5

    accuracy                           0.15        53
   macro avg       0.44      0.44      0.15        53
weighted avg       0.73      0.15      0.15        53

✳️ Entrenando modelo Ejex_3_64_4

⏺️ Épocas 20 | LR-WD 0.005-0.0005 | Patience 3
▶️ Start train


...

>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.6955 ||BAC =0.5345 | Recall =1.0000 | AUC=0.4828


>> Mejor modelo guardadoEpoch 3/20 | TrainLoss=0.6600 ||BAC =0.5483 | Recall =0.2000 | AUC=0.3759


>> Early stopping activado 5
Tiempo total de entrenamiento: 3.73 minutos
Best model: ../Models_Output/Ejex_15_64_4_20_2_3.pth

 TEST =====
BAC = 0.5750 | AUC = 0.5750 | recall = 0.4000  | f1 = 0.2105

Matriz de confusión:
[[36 12]
 [ 3  2]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.92      0.75      0.83        48
    Dementia       0.14      0.40      0.21         5

    accuracy                           0.72        53
   macro avg       0.53      0.57      0.52        53
weighted avg       0.85      0.72      0.77        53

✳️ Entrenando modelo Ejex_16_64_4

⏺️ Épocas 20 | LR-WD 0.0001-0.0001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.7169 ||BAC =0.5000 | Recall =0.0000 | AUC=0.4655


>> Early stopping activado 3
Tiempo total de entrenamiento: 2.44 minutos
Best model: ../Models_Output/Ejex_16_64_4_20_0_3.pth

 TEST =====
BAC = 0.4896 | AUC = 0.4896 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[47  1]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.90      0.98      0.94        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.89        53
   macro avg       0.45      0.49      0.47        53
weighted avg       0.82      0.89      0.85        53



### 8

In [16]:
n_slices = 64
BATCH = 8
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_64_8

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.0376 ||BAC =0.5000 | Recall =1.0000 | AUC=0.5000


>> Early stopping activado 3
Tiempo total de entrenamiento: 1.83 minutos
Best model: ../Models_Output/Ejex_1_64_8_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 1.0000  | f1 = 0.1724

Matriz de confusión:
[[ 0 48]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.00      0.00      0.00        48
    Dementia       0.09      1.00      0.17         5

    accuracy                           0.09        53
   macro avg       0.05      0.50      0.09        53
weighted avg       0.01      0.09      0.02        53

✳️ Entrenando modelo Ejex_2_64_8

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.0077 ||BAC =0.4672 | Recall =0.9000 | AUC=0.5655


>> Mejor modelo guardadoEpoch 2/20 | TrainLoss=0.7494 ||BAC =0.5931 | Recall =0.6000 | AUC=0.5828


...

>> Early stopping activado 3
Tiempo total de entrenamiento: 2.04 minutos
Best model: ../Models_Output/Ejex_15_64_8_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 0.0000  | f1 = 0.0000

Matriz de confusión:
[[48  0]
 [ 5  0]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.91      1.00      0.95        48
    Dementia       0.00      0.00      0.00         5

    accuracy                           0.91        53
   macro avg       0.45      0.50      0.48        53
weighted avg       0.82      0.91      0.86        53

✳️ Entrenando modelo Ejex_16_64_8

⏺️ Épocas 20 | LR-WD 0.0001-0.0001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.7109 ||BAC =0.4552 | Recall =0.6000 | AUC=0.5000


>> Mejor modelo guardadoEpoch 2/20 | TrainLoss=0.6676 ||BAC =0.4586 | Recall =0.4000 | AUC=0.4414


>> Mejor modelo guardadoEpoch 4/20 | TrainLoss=0.5081 ||BAC =0.6034 | Recall =1.0000 | AUC=0.3276


>> Early stopping activado 6
Tiempo total de entrenamiento: 3.78 minutos
Best model: ../Models_Output/Ejex_16_64_8_20_3_3.pth

 TEST =====
BAC = 0.5312 | AUC = 0.5312 | recall = 1.0000  | f1 = 0.1818

Matriz de confusión:
[[ 3 45]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       1.00      0.06      0.12        48
    Dementia       0.10      1.00      0.18         5

    accuracy                           0.15        53
   macro avg       0.55      0.53      0.15        53
weighted avg       0.92      0.15      0.12        53



### 16

In [17]:
n_slices = 64
BATCH = 16
# Ejecutar cada combinación
for i, (LR, WEIGHT) in enumerate(grid, start=1):
    MODELO = f"Ejex_{i}_{n_slices}_{BATCH}"
    main(df, DEVICE, n_slices, EPOCHS, LR, WEIGHT, EARLY, BATCH, MODELO)

✳️ Entrenando modelo Ejex_1_64_16

⏺️ Épocas 20 | LR-WD 0.005-0.005 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.1092 ||BAC =0.5000 | Recall =1.0000 | AUC=0.5000


>> Early stopping activado 3
Tiempo total de entrenamiento: 2.36 minutos
Best model: ../Models_Output/Ejex_1_64_16_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 1.0000  | f1 = 0.1724

Matriz de confusión:
[[ 0 48]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.00      0.00      0.00        48
    Dementia       0.09      1.00      0.17         5

    accuracy                           0.09        53
   macro avg       0.05      0.50      0.09        53
weighted avg       0.01      0.09      0.02        53

✳️ Entrenando modelo Ejex_2_64_16

⏺️ Épocas 20 | LR-WD 0.005-0.001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=1.1800 ||BAC =0.5000 | Recall =1.0000 | AUC=0.5000


>> Early stopping activado 3
Tiempo total de entrenamiento: 2.37 minutos
Best model: ../Models_Output/Ejex_2_64_16_20_0_3.pth

 TEST =====
BAC = 0.5000 | AUC = 0.5000 | recall = 1.0000  | f1 = 0.1724

Matriz de confusión:
[[ 0 48]
 [ 0  5]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.00      0.00      0.00        48
    Dementia       0.09      1.00      0.17         5

    accuracy                           0.09        53
   macro avg       0.05      0.50      0.09        53
weighted avg       0.01      0.09      0.02        53

✳️ Entrenando modelo Ejex_3_64_16

⏺️ Épocas 20 | LR-WD 0.005-0.0005 | Patience 3
▶️ Start train


...

>> Mejor modelo guardadoEpoch 3/20 | TrainLoss=0.5457 ||BAC =0.5500 | Recall =0.1000 | AUC=0.4621


>> Early stopping activado 5
Tiempo total de entrenamiento: 3.58 minutos
Best model: ../Models_Output/Ejex_15_64_16_20_2_3.pth

 TEST =====
BAC = 0.7688 | AUC = 0.7688 | recall = 0.6000  | f1 = 0.5455

Matriz de confusión:
[[45  3]
 [ 2  3]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.96      0.94      0.95        48
    Dementia       0.50      0.60      0.55         5

    accuracy                           0.91        53
   macro avg       0.73      0.77      0.75        53
weighted avg       0.91      0.91      0.91        53

✳️ Entrenando modelo Ejex_16_64_16

⏺️ Épocas 20 | LR-WD 0.0001-0.0001 | Patience 3
▶️ Start train


>> Mejor modelo guardadoEpoch 1/20 | TrainLoss=0.8114 ||BAC =0.5000 | Recall =1.0000 | AUC=0.4759


>> Mejor modelo guardadoEpoch 3/20 | TrainLoss=0.5601 ||BAC =0.5328 | Recall =0.1000 | AUC=0.4586


>> Early stopping activado 5
Tiempo total de entrenamiento: 3.58 minutos
Best model: ../Models_Output/Ejex_16_64_16_20_2_3.pth

 TEST =====
BAC = 0.7167 | AUC = 0.7167 | recall = 0.6000  | f1 = 0.3750

Matriz de confusión:
[[40  8]
 [ 2  3]]

Reporte de clasificación:
              precision    recall  f1-score   support

 No Dementia       0.95      0.83      0.89        48
    Dementia       0.27      0.60      0.38         5

    accuracy                           0.81        53
   macro avg       0.61      0.72      0.63        53
weighted avg       0.89      0.81      0.84        53



## Resumen

____________________

Solo 24 modelos superaron un BAC de 0.60 de los 144 probados.

Se comprobó que:
* 64 slices es el que produce mejores modelos
* Batch 8 es el size óptimo
* Lr y Weight decay óptimos se encuentran entre 0.001 y 0.0005